<a href="https://colab.research.google.com/github/Meaveryway/Backend_AntTech/blob/master/5_Abstract_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install langdetect
!pip install tika
!pip install pdfplumber

from langdetect import detect
import time
from datetime import datetime
import csv
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os.path
import re
import nltk

import pdfplumber
from tika import parser
from bs4 import BeautifulSoup
from io import StringIO
from tika import detector


from time import sleep
from random import randint
import dateutil.parser as dateParser
import concurrent.futures

     |████████████████████████████████| 983kB 4.0MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993223 sha256=b67c98bdcc46660898cb3e46ef4560b96c4a96ed875545adbbbb26d8ac9ddd60
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
Successfully built langdetect
  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=45c815b47bd3392f357e03aad34e33bd812284e2927596a9b4ee943507b86225
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika
     |████████████████████████████████| 51kB 2.1MB/s 
     |████████████████████████████████| 5.6MB 4.3MB/s 
     |████████████████████████████████| 143kB 38.5MB/s 
     |████████████████████████████████| 1.9MB 39.2MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.28-cp37-none-any.whl size=32222 sha256=49b8925df04d85414e7155b34d3861ad41a907a2a042b089949ea0e

In [ ]:
!pip freeze > requirements.txt

In [ ]:
!pip install elastic_enterprise_search==7.10.0a1
from elastic_enterprise_search import AppSearch

In [ ]:
#context = ssl._create_unverified_context()
import requests
requests.packages.urllib3.disable_warnings()

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [ ]:
# Dataframes en mode interactif
%load_ext google.colab.data_table
#%unload_ext google.colab.data_table

# Access To Google Drive
In order to write files on there for persistency.

In [9]:
DRIVE_PATH = "/content/drive/MyDrive/Théses-Algérie/Data"
MISSED_LINKS_PATH = DRIVE_PATH + "missingno"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Connecting to the Engine 

In [ ]:
# Connecting to an instance on VPS w/ an App Search private key
app_search = AppSearch(
    "https://www.theses-dz.ml:8080",
    http_auth="private-vq7y3ap5tkax1r7wouo2xzfm",
    ca_certs=False,
    verify_certs=False,
)
engine_name = "theses-dz"
engine_name_full_text = "full-text-sampler"
app_search.get_schema(engine_name)

/usr/local/lib/python3.7/dist-packages/elastic_transport/connection/http_urllib3.py:183: UserWarning: Connecting to 'https://www.theses-dz.ml:8080' using SSL with verify_certs=False is insecure
  % self.base_url


{'keywords': 'text', 'additional_urls': 'text', 'body_content': 'text', 'domains': 'text', 'language': 'text', 'abstract': 'text', 'title': 'text', 'type': 'text', 'body': 'text', 'meta_keywords': 'text', 'url': 'text', 'meta_description': 'text', 'field': 'text', 'publication_date': 'number', 'publisher': 'text', 'links': 'text', 'identifier_uri': 'text', 'authors': 'text'}

---
---
---

# Single indexation (manual)

In [ ]:
TITLE = ''
ID = int(str(int(hashlib.md5(str(TITLE).encode('utf-8')).hexdigest(), 16))[:16])
LINK = ''
PUBLISHER = ''
ABSTRACT = ""

AUTHORS = ['', '']
keywords = ['', '']

YEAR = 0
LANGUAGE = ''

FIELD = ''
TYPE = ''

FILE_URL = ""

#------------------------------------------
file_to_index={
          "id": ID,
          "title": TITLE,
          "abstract": ABSTRACT,
          "authors": AUTHORS,
          "publication_date": YEAR,
          "field": FIELD,
          "keywords": KEYWORDS,
          "type": TYPE,
          "publisher": PUBLISHER,
          "url": FILE_URL,
          "identifier_uri": LINK,
          "language": LANGUAGE,
          #"body": ""
      }]

In [ ]:
app_search.index_documents(engine_name, file_to_index)

---
---
---
###### Some patchwork

# File Collection & Summarization


In [39]:
def digitize_file_pdfplumber(file_path):
  text = ""
  with pdfplumber.open(file_path) as pdf:
    print("ok")
    pages = pdf.pages
    for page in pages:
      text = text + page.extract_text()
  pdf.close()
  return(text)
  

def digitize_file_tika(file_path):
    file_extension = detector.from_file(file_path)
    if 'pdf'in str(file_extension).lower():
      print("Parsing PDF")
      file_data = []
      buffer = StringIO()
      data = parser.from_file(file_path, xmlContent=True)
      xhtml_data = BeautifulSoup(data['content'], features="lxml")
      for page, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
          #print('Parsing page {} of pdf file...'.format(page+1))
          buffer.write(str(content))
          parsed_content = parser.from_buffer(buffer.getvalue())
          buffer.truncate()
          file_data.append({'id': str(page+1), 'content': parsed_content['content'].replace('\n\n', '').replace('\n', ' ')})
      #Clean-up
      for i in reversed(range(1,len(file_data))) :
        content = file_data[i]['content']
        substract = file_data[i-1]['content']
        modified_content = content.replace(substract, "")
        file_data[i]['content'] = modified_content
        file_data[i] = file_data[i]['content']
      return file_data
    
    else:
      text = data = parser.from_file(file_path)['content']
      return(text)

def digitize_file(file_path):
  file_extension = detector.from_file(file_path)
  if 'pdf'in str(file_extension).lower():
    print("going for Plumber")
    fulltext = digitize_file_pdfplumber(file_path)
  else:
    fulltext = digitize_file_tika(file_path)
    print("going for Tika")
  return fulltext

def download_file(download_url, filename):
    r = requests.get(download_url, stream = True) 
    with open(filename,"wb") as pdf: 
      for chunk in r.iter_content(chunk_size=1024): 
        # One chunk at a time
        if chunk: 
          pdf.write(chunk) 

In [ ]:
def index_fulltext(item):
    id = item["id"]['raw']

    #check if file already indexed

    result = app_search.search(
        engine_name=engine_name,
        body={
            "query": "",
            "page": {
                "size": 10,
            },
            "filters" : {
                "all":[
              {"id": '7859947258069749'},
              ]
            }
        }
    )


    title = item['title']['raw']
    authors = item['authors']['raw']
    field= item['field']['raw']
    abstract = item['abstract']['raw']
    publication_date = item['publication_date']['raw']
    language = item['language']['raw']
    keywords = item['keywords']['raw']
    identifier_uri = item['identifier_uri']['raw']
    publisher = item['publisher']['raw']
    doc_type = item['type']['raw']
    url = item['url']['raw']
    print(title)

      
    download_file(url, str(id))
    print("file downloaded " + url)
    raw_text = digitize_file('/content/' + str(id))
    try:
      raw_text[0] = raw_text[0]['content']
    except:
      pass
    raw_text = ''.join(raw_text)
    print("file parsed")
    #print(raw_text)
    app_search.index_documents(
        engine_name=engine_name_full_text,
        body=[{
              "id": id,
              "title": title,
              "abstract": abstract,
              "authors": authors,
              "publication_date": publication_date,
              "field": field,
              "keywords": keywords,
              "type": doc_type,
              "publisher": publisher,
              "url": url,
              "identifier_uri": identifier_uri,
              "language": language,
              "body": raw_text
        }]
    )

In [ ]:
yearly_results = app_search.search(
        engine_name=engine_name,
        body={
            "query": "",
            "page": {
                "size": 1000,
            },
            "filters" : {
                "all":[
              {"publication_date": 2020},
              {"publisher": "Université Mohamed Boudiaf - M'sila"}]
            }
        }
    )
results = yearly_results['results']
print(len(results))

1000


In [ ]:
for result in results[250:]:
  try:
    index_fulltext(result)
  except:
    print("Failed to index file for =={}== : {}".format(result['title']['raw'], result['url']['raw']))
    pass

In [ ]:
result = app_search.search(
        engine_name=engine_name,
        body={
            "query": "",
            "page": {
                "size": 10,
            },
            "filters" : {
                "all":[
              {"id": '7859947258069749'},
              ]
            }
        }
    )


In [ ]:
result['meta']['page']['total_results'] > 0

True

In [ ]:
result['results'][0]

#### Summerization

In [3]:
import nltk
from nltk.corpus import stopwords
from itertools import chain
from heapq import nlargest
nltk.download('stopwords')

stop_words = stopwords.words('french')
stop_words += ['comme', 'chapitre', 'peut', 'être', 'avoir']

from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')
from string import punctuation
punctuation = punctuation + '\n'
punctuation

exclusion_set = set(chain(punctuation, stop_words))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [102]:
def get_word_frequencies(text):
  words = word_tokenize(text)
  word_frequencies = {}
  for word in words:
    if word.lower() not in exclusion_set:
      if word.lower() not in word_frequencies.keys():
        word_frequencies[word.lower()] = 1
      else:
        word_frequencies[word.lower()] += 1
  
  max_frequency = max(word_frequencies.values())
  for word in word_frequencies.keys():
    word_frequencies[word] /= max_frequency
  return word_frequencies

def get_sentence_weights (text, word_frequencies):
  sentences = sent_tokenize(text)
  sentence_weights = dict()
  for sentence in [x for x in sentences if len(x) > 5]:
    
    weight = 0
    tokens = word_tokenize(sentence.lower())
    clean_tokens = [token.lower() for token in tokens if token.lower() not in exclusion_set and token.isalpha()]
    #print(clean_tokens)
    if len(clean_tokens) > 1:
      bigrams = nltk.bigrams(clean_tokens)
      #word_count = len(sentence - exclusion_set)
      for gram in bigrams:
        try:
          weight += word_frequencies[gram]
        except:
          pass
      if sentence.lower() not in sentence_weights.keys():
        sentence_weights[sentence.lower()] = weight #/word_count
  return sentence_weights

def generate_summary(text):
  #word_freqs = get_word_frequencies(text)

  tokens = word_tokenize(raw_text)
  clean_tokens = [token.lower() for token in tokens if token.lower() not in exclusion_set and token.isalpha()]
  grams = nltk.bigrams(clean_tokens)

  word_freqs = nltk.FreqDist(grams)

  sent_weights = get_sentence_weights(text, word_freqs)
  select_length = min(int(len(sent_weights)*0.05) + 1, 8)
  print("Select length for summary = {} sentences".format(select_length))
  summary = nlargest(select_length, sent_weights, sent_weights.get)
  ordered_summary = []

  for sent in sent_weights.keys():
    if sent in summary:
      ordered_summary.append(' '.join(sent.split()))

  summary = '\n - '.join(ordered_summary)

  keywords = []#nlargest(10, [word.lower() for word in list(word_freqs.keys()) if word.lower() not in exclusion_set and word.isalpha()], word_freqs.get)
  return summary, keywords

def generate_keywords(text, number=10):
  tokens = word_tokenize(raw_text)
  clean_tokens = [token.lower() for token in tokens if token.lower() not in exclusion_set and token.isalpha()]
  grams = nltk.bigrams(clean_tokens)

  gram_freqs = nltk.FreqDist(grams)
  sorted_gram_freqs = {k: v for k, v in sorted(gram_freqs.items(), reverse =True, key=lambda item: item[1])}
  keywords = dict((k,v) for k,v in sorted_gram_freqs.items())
  keywords = [*keywords.keys()]

  return [' '.join(combined) for combined in keywords[:number]]


- Source file: https://www.theses-algerie.com/2752754995528827/memoire-de-master/universite-mohamed-boudiaf---m-sila/l’analyse-sémiotique-et-comparative-des-accessoires-algériens-,-cas-des-bijoux-kabyles-et-des-bijoux-nailis?size=n_10_n

Author's keywords: sémiotique, bijoux traditionnels, communication verbale, kabylie, oulennail, formes, couleurs

In [116]:
def curves_to_edges(cs):
    """See https://github.com/jsvine/pdfplumber/issues/127"""
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

def not_within_bboxes(obj):
    """Check if the object is in any of the table's bbox."""
    def obj_in_bbox(_bbox):
        """See https://github.com/jsvine/pdfplumber/blob/stable/pdfplumber/table.py#L404"""
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)
    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)



text = ""
with pdfplumber.open("/content/12345678") as pdf:
    print("ok")
    pages = pdf.pages
    for page in pages[10:len(pages)-10]:
      crop_coords = [0, .1, 1, .85]

      my_width = page.width
      my_height = page.height
        # Crop pages
      my_bbox = (crop_coords[0]*float(my_width), crop_coords[1]*float(my_height), crop_coords[2]*float(my_width), crop_coords[3]*float(my_height))
      
      ts = {
      "vertical_strategy": "explicit",
      "horizontal_strategy": "explicit",
     "explicit_vertical_lines": curves_to_edges(page.curves + page.edges),
     "explicit_horizontal_lines": curves_to_edges(page.curves + page.edges),
    "intersection_y_tolerance": 10,
      }
      page = page.crop(bbox=my_bbox)
      bboxes = [(image['x0'], image['y0'], image['x1'], image['y1']) for image in page.images] + [table.bbox for table in page.find_tables(table_settings=ts)]
      text = text + page.filter(not_within_bboxes).extract_text().strip()
pdf.close()

ok


In [117]:
raw_text = text.replace('\n', '').strip()

In [119]:
  std_word_freqs = get_word_frequencies(raw_text)
  
  tokens = word_tokenize(raw_text)
  clean_tokens = [token.lower() for token in tokens if token.lower() not in exclusion_set and token.isalpha()]
  grams = nltk.bigrams(clean_tokens)

  word_freqs = dict(nltk.FreqDist(grams))
  max_frequency = max(word_freqs.values())
  for word in word_freqs.keys():
    word_freqs[word] /= max_frequency
  print(word_freqs)
  sent_weights = get_sentence_weights(raw_text, word_freqs)
  select_length = max(int(len(sent_weights)*0.15) + 1, 12)
  print("Select length for summary = {} sentences".format(select_length))
  summary = nlargest(select_length, sent_weights, sent_weights.get)
  ordered_summary = []

  for sent in sent_weights.keys():
    if sent in summary:
      ordered_summary.append(' '.join(sent.split()))

  summary = '\n - '.join(ordered_summary)
  print(summary)
  #keywords = nlargest(10, [word.lower() for word in list(word_freqs.keys()) if word.lower() not in exclusion_set and word.isalpha()], word_freqs.get)
  

{('mieux', 'comptons'): 0.043478260869565216, ('comptons', 'adopter'): 0.043478260869565216, ('adopter', 'approche'): 0.043478260869565216, ('approche', 'sémiotique'): 0.043478260869565216, ('sémiotique', 'menée'): 0.043478260869565216, ('menée', 'aide'): 0.043478260869565216, ('aide', 'méthode'): 0.043478260869565216, ('méthode', 'descriptive'): 0.043478260869565216, ('descriptive', 'analytique'): 0.043478260869565216, ('analytique', 'comparative'): 0.043478260869565216, ('comparative', 'estimons'): 0.043478260869565216, ('estimons', 'convenable'): 0.043478260869565216, ('convenable', 'cette'): 0.043478260869565216, ('cette', 'étude'): 0.043478260869565216, ('étude', 'travail'): 0.043478260869565216, ('travail', 'subdivisé'): 0.043478260869565216, ('subdivisé', 'deux'): 0.043478260869565216, ('deux', 'parties'): 0.043478260869565216, ('parties', 'théorique'): 0.043478260869565216, ('théorique', 'pratique'): 0.043478260869565216, ('pratique', 'première'): 0.043478260869565216, ('premiè

In [103]:
#url = "http://dspace.univ-msila.dz:8080/xmlui/bitstream/handle/123456789/21897/fr-77-2020.pdf?sequence=1&isAllowed=y"
#sample = download_file(url, '12345678')
#raw_text = digitize_file('12345678')
try:
  raw_text[0] = raw_text[0]['content']
except:
  pass
raw_text = ''.join(raw_text)


summary, one_word_keywords = generate_summary(text)
bigram_keywords = generate_keywords(raw_text)

going for Plumber
ok
Select length for summary = 8 sentences


In [104]:
print(summary)
print('----------------')
print(', '.join(one_word_keywords))
print('----------------')
print(', '.join(bigram_keywords))
print('----------------')
print("Author's keywords: sémiotique, bijoux traditionnels, communication verbale, kabylie, oulennail, formes, couleurs")

république algérienne démocratique et populaire ministère de l’enseignement supérieur et de la recherche scientifique université mohamed boudiaf - m’sila faculté des lettres et des langues domaine : lettres et langues étrangères département des lettres et langue française filière : langue française n° : …………………………………… option : sciences du langage mémoire présenté pour l’obtention du diplôme de master académique par : mlle.fenniche meyssa intitulé : l’analyse sémiotique et comparative des accessoires algériens , cas des bijoux kabyles et des bijoux nailis soutenu devant le jury composé de : zaghba linda sité de m’sila président boudjellelnadjia université de m’sila rapporteurrépublique algérienne démocratique et populaire ministère de l’enseignement supérieur et de la recherche scientifique université mohamed boudiaf - m’sila faculté des lettres et des langues domaine : lettres et langues étrangères département des lettres et langue française filière : langue française n° : ………………………………

##### Original keywords:
linguistique, français, langage, réseau sociaux, nouveaux mots utilisés, conversations, jeunes utilisateurs, Facebook
##### Generated keywords
[(('arabe', 'algérien'), 31), (('langue', 'française'), 25), (('sociolinguistique', 'algérie'), 25), (('situation', 'sociolinguistique'), 23), (('réseaux', 'sociaux'), 22), (('analyse', 'corpus'), 22), (('arabe', 'dialectal'), 22), (('alternance', 'codique'), 15), (('jeunes', 'algériens'), 14), (('arabe', 'classique'), 12), (('situation', 'linguistique'), 11)


In [ ]:
import inspect
lines = inspect.getsource(get_word_frequencies)
print(lines)

def get_word_frequencies(text):
  words = word_tokenize(text)
  word_frequencies = {}
  for word in words:
    if word.lower() not in exclusion_set:
      if word.lower() not in word_frequencies.keys():
        word_frequencies[word.lower()] = 1
      else:
        word_frequencies[word.lower()] += 1
  
  max_frequency = max(word_frequencies.values())
  for word in word_frequencies.keys():
    word_frequencies[word] /= max_frequency
  return word_frequencies



---
---
## Abstractive summarization of extracted sentences: 
https://huggingface.co/plguillou/t5-base-fr-sum-cnndm

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
  
tokenizer = T5Tokenizer.from_pretrained("plguillou/t5-base-fr-sum-cnndm")
model = T5ForConditionalGeneration.from_pretrained("plguillou/t5-base-fr-sum-cnndm")

In [39]:
sentences = [
"elle a dit également que « la notion du signe , elle existe déjà , elle ne concerne pas seulement le langage médical mais apparait aussi dans la philosophie du langage tell qu’on peut le lire chez platon et plus particulièrement chez aristote […] »5 i.1 la sémiologie : la sémiologie est une discipline d’origine européenne , elle est née au début du vingtième siècle et grâce au linguiste, ferdinand de saussure ‘’ le père fondateur de la linguistique et de la sémiologie ‘’ .",
"signe peut d’abord être un équivalent d’indice ; l’indice ( ou signe ) est un phénomène , le plus souvent naturel , immédiatement perceptible , qui nous fait connaitre quelque chose au sujet d’un autre phénomène non immédiatement perceptible »9 le signe se retrouve dans notre vie quotidienne , nous le voyons comme ( une couleur , un objet , les gestes …) pour présenter quelque chose d’autre abstraite .",
"7ferdinand de saussure , 2002 , « cours de linguistique générale » , bejaia : talantikit , p 22.signe comme « un objet matériel , figure ou son perceptible tenant lieu d’une chose absente ou impossible de percevoir , et servant soit à la rappeler à l’esprit ,soit à se combiner avec d’autres signes pour effectuer une opération .»10 i.1.1.2 le signe linguistique: pour ferdinand de saussure le signe linguistique est définit comme une entité psychique à deux faces : le signifié et le signifiant , en d’autre terme « le signe linguistique unit non une chose et un nom , mais un concept et une image »11 signifie concept le signe linguistique signifiant image acoustique figure1 : le signe saussurien  le signifié ( le concept ) pour f.de saussure c’est la face psychique du signe linguistique , il représente l’idée et non pas la suite de sons au contraire du signifiant.",
 "11ferdinand de saussure , « cours de linguistique générale », 2002, bejaia , talantikit , p 85.i.1.1.3 les principes du signe linguistique: le signe linguistique se caractérise par deux principes fondamentaux: i.1.1.3.1 l’arbitraire du signe: le signe linguistique se repose sur deux phases le signifié et le signifiant .ces deux se réunissent par des liens sociaux et la relation entre eux est conventionnelle , autrement dit ce qui relie le signifiant et le signifié n’a pas un lien logique c’est effectivement arbitraire .",
 "- comme il l’affirme saussure : « le lient unissant le signifiant au signifie est arbitraire ou encore , puisque nous entendons par signe le totale résultat de l’association d’un signifiant à un signifie , nous pouvons dire plus simplement : le signe linguistique est arbitraire »13 i.1.1.3.2 caractère linéaire : cecaractère concerne le signifiant , d’après saussure « le signifiant , étant de nature auditive se déroule dans le temps seul et a les caractères qu’il emprunte : a) il représente une étendue , et b) cette étendue est mesurable dans une seule dimension : c’est la ligne .",
" - »19 i.2.2 le signe chez charles sander peirce: pour peirce « un signe est quelque chose qui tient lieu pour quelqu’un de quelque chose sous quelque rapport ou à quelque titre » , et selon lui « […] il considérait le signe comme un objet , une entité à trois termes : un représentamen , un objet et interprétant .»20 d’après cette définition , nous constatons que le signe intègre toutes sortes de matérialités et plus précisément le signe dépond la conception de l’être ; ainsi que le signe est composé par la relation de trois composants :le représentamen, c'est le signifiant chez saussure; l'interprétant,c'est le signifié chez saussure et en fin le référent qui est l'objet .",
 "- i.2.2.1 les composants du signe chez peirce: peirce nous explique qu’une triade relationnelle permet de comprendre les signes dans trois éléments :interprétant représentamen objet figure n°2 : le signe peircien i.2.2.1.1 le représentamen: c’est ce qui est perceptible d’un signe ; d’autre terme c’est quelque chose qui représente une autre chose : un mot , un logo , un symbole .",
" - piero nardi a défini l’objet dans son colloque comme suit « […] l’objet se définit alors comme ce qui est fabriqué, c’est de la matière finie standardisée, formée, et normalisée[…] le paradoxe que je voudrais signaler, c’est que ces objets qui ont toujours en principe une fonction, une unité, un usage, nous croyons les vivre comme des instruments purs ; alors qu’en réalité ils véhiculent d’autres choses ils sont aussi autres choses : ils véhiculent du sens, autrement dit, l’objet sert effectivement à quelque chose, mais il sert aussi communiquer des informations ce que nous pourrions résumer d’une phrase en disant qu’il y a toujours un sens qui déborde l’usage de l’objet»26 i.4 la sémiologie de la communication : la sémiologie de la communication est l’un des écoles fondamentaux de la sémiologie , elle étudie uniquement le mode de signes .",
" - alors , la sémiotique de la culture se définit comme « science qui étudie tous les phénomènes de culture comme s’ils étaient des signes en se basant sur l’hypothèse que tous les phénomènes sont en réalité des systèmes de signes »36 son champ d’étude « recouvre la sémiotique de l’identité dans ses deux dimensions individuelle et collective qui se rattache à la famille ou au groupe dont les traditions , les façons de s’habiller , de faire sa toilette , de se nourrir en constitue un mode d’expression »37 33albert dauzat et al , nouveau dictionnaire etymologie et historique , etymologie , paris , larousse , 1971 , p ,682en effet , la sémiotique de la culture selon eco est venue comme une suite de la sémiologie de la communication « la culture n'ayant pas seulement une fonction de communication, mais nous ne pouvons comprendre la culture qu'à travers son aspect communicatif, et par conséquent, nous pouvons évidemment constater le lien établi entre les lois de la communication et celles de la culture [...], les lois des systèmes sémiotiques sont des lois culturels .",
" - a partir de ce chapitre, nous comprenons que le langage humain et la communication qui sont faits entre les individus ne dépondent pas uniquement des mots , mais aussi à travers l’image , les bijoux , les codes vestimentaires .. la sémiologie s’intéresse non seulement aux signes mais elle étudie également l’image sous l’angle de la signification pour dégager son rôle communicatif .les aspects culturels et géographiques en algérie: en kabylie et à oulednailintroduction dans ce chapitre , nous mettons la lumière sur la culture algérienne en abordant en premier lieu l’étymologie et la définition du mot culture , aussi bien les définitions des anthropologues que les philosophes , puis nous passons à la culture algériennes en générale en donnant la géographie de l’algérie , ses richesses naturelles , ses vestiges anciens.",
" - »39cela signifie qu’il n’est pas ni culture sans langage , ni langage sans culture c’est-à-dire qu’il y a une relation complémentaire entre les deux .1.2.3 autres définitions de la culture : selon le sociologue québécois guy rocher , elle est : « un ensemble lié de manière de penser , de sentir , et d’agir plus ou moins formalisées qui étant apprises et partagées par une pluralité de personnes , servent d’une manière à la fois objective et symbolique , à constituer ces personnes en une collectivité particulière et distincte .»40 selon le philosophe edgar morin : «la culture est un patrimoine informationnel constitués des savoirs , savoir-faire , règles , normes propres à une société [ … ].",
" - le burnous est la tenue essentielle dans les vêtements masculins , il est sous la forme d’un manteau en laine ou bien en soie avec une couleur blanche ou marron ; il est porté pour préserver le corps de l’homme contre le froid , le burnous est porté durant les fêtes traditionnelles ( les aïds , mariages , yennayer pour les kabyles … ).représente en générale la femme kabyle , l’identité et la culture amazigh ( berbère ) ; la robe kabyle traditionnelle se caractérise par ses différentes pièces , ses couleurs spécifiques , elle a eu un modèle standard et unique malgré tous les modifications à travers le temps .« c’est une robe de laine constituée de deux rectangles assemblés par une couture sur les côtés etd’unempiècementdoublésanscoutureàl’épaule,ilcomprenddesmancheslongues .l’encolure et souvent coupée en v ou une fente jamais tors large ou profonde .le corsage et le bas de la robe sont richement décorés de galons de couleurs vif .»45 elle se compose de cinq éléments : la robe kabyle qui est la pièce essentielle fabriquée à la cotonnade ou à la soie tissée avec des différents motifs berbères , le foulard est également tissée aves des motifs berbères des différentes couleurs , la fouta brodée en soie elle se met à la partie inférieure du corps de la femme , aussi bien la ceinture qui est un ensemble de fils en ém il 3.2.2 la poterie kabyle: la poterie kabyle est l’un des points essentielle et important des traditions de la kabylie , en d’autre terme elle est l’un des piliers de l’artisanat traditionnelle , elle est originale par ses formes et ses motifs .",
" - « en ce sens , la poterie féminine algérienne nous raconte , d’abord , histoire de femme inscrites dans un contexte historique , technologique , social , et culturel bien défini et chargé , par les groupes , d’une responsabilité précise , celle d’assurer le bien-être familial , et d’abord l’eau et le pain , elle nous raconte aussi l’histoire de femme qui, en dépit de la lourdeur de cette responsabilité et des contraintes qui en découlent , parviennent à se réserver , à s’approprier un territoire d’imagination , de rêve et de créativité , ou elles peuvent se penser comme des individus»46.",
" - 3.2.3 les bijoux kabyles: les bijoux ont toujours inclut la vie féminine dans le monde , chaque région a un genre distinct de bijoux l'exemple des bijoux kabyles ; ceux-ci feront l'objet de notre analyse .",
" - « les bijoux traditionnels de la kabyle du djurdjura sont à base d’argent , parfois des pièces anciennes appelé lwiz , revêtues entièrement d’un décor d’émaux vitrifies , multicolores , verts, bleus ou jaunes , cloisonnés par des filigranes , ornés de cabochons de corail rouge ( souvent par cinq ) et de nombreuses perles d’argent que l’on appelle irden ( grains de blé ) , avec le plus souvent , des pendeloques .»47 d’après cette citation , nous constatons que les bijoux kabyles sont fabriqués en argent , ce dernier est la base de l’industrie du bijou , ils sont décorés par de différentes couleurs d’émaux , ils ont de différentes formes .",
" - pour les tenues masculines , il existe deux genres des tenus traditionnels qui sont importants pour oulednail : burnous et kachabia ; pour le burnous nous avons déjà parlé sur lui dans les pages précédentes , tandis que , il est remarquable que le tissage de kachabia occupe une place précieuse dans le cœur d’artisanat nailie, elle est fabriquée en pure laine de mouton ou avec l’oubar , autrement dit les poiles des chameaux , la kachabia est connue pour protéger du froid , elle ressemble à un ample manteau .ce travail demande beaucoup de savoir-faire surtout pour la broderie et les ornements.",
" - les bijoux nailis font partie de la culture nailie, ils sont associés avec les tenues féminines , portés dans les fêtes traditionnelles comme les mariages, les aïds… la femme nailie a pris en soin sa beauté et sa forme , elle est connue depuis longtemps par son amour des bijoux ; alors que la femme nailie a acquis ces bijoux pour les jours difficiles en confirmant le proverbe populaire « hdayedlchdaid » .",
" - la culture algérienne est très riche par son patrimoine diversifié, elle est riche par sa population qui gardent et préservent ce patrimoine culturel , de là, nous pouvons dire que chaque nation a un héritage dont elle est fière , car c'est la racine qui s'étend dans un passé lointain, la nation avec sa grande gloire, et l’algérie comme les autres pays du monde possède une diversité culturelle distincte, qui doit être respectée et préservée afin de rester gravée dans la mémoire de nos générationsfutures.description et analyse du corpus : notre corpus se présente sous formes d'images , il est construit de photos tirés des sites internet comme pinterest , et de réseaux sociaux comme facebook et instagram.",
" - description de chaque échantillon : a_ les bijoux kabyles: il s’agit d’un ensemble d’accessoires riche par leurs différentes couleurs , elles représentent aussi la région de la kabylie qui est connue par son originalité et ses traditions , telles que bejaia , tiziouzou , bordj bou arreridj … à travers laquelle nous cherchons à mettre en évidence et à dégager les symboles sociaux et les connotations que ces bijoux portent pour les habitants de cette région .",
" - en plus , les bijoux kabyles comme les bijoux nailis dans leur utilisation ont leur valeur matérielle et significative qui détermine leur fonction dans la société , ainsi que leurs différents usages: 3.1 usage esthétique: les bijoux sont le monde magique des femmes , leurs formes sont très variés pour donner l’élégance , la beauté et le charme à celle qui les porte .",
" - tableau n°1:signification de la couleur rouge5.2 le jaune : le jaune est une couleur primaire chaude , pleine de vie et d’énergie qui représente la joie , la tonicité et la dynamisme ; elle peut signifier aussi la douceur , la richesse ; c’est la couleur du soleil et de l’or .",
" - tableau n° 2 : signification de la couleur jaune 5.3 le bleu: le bleu est une couleur primaire froide , elle nous rappelle à l’infini puisqu’elle nous fait penser à la mer et le ciel .",
" - tableau n°3 : signification de la couleur bleue 5.4 le vert: le vert est une couleur primaire froide , qui fait penser à la nature , elle symbolise l’équilibre , la permission ; elle peut signifier aussi le bonheur , la réussite , l’énergie et la jeunesse .",
" - tableau n°9 : les noms des bijoux kabyles : tableau n°10 : les noms des bijoux nailis7.2 description et interprétation de chaque bijou: 7.2.1 les bijoux kabyles: diadème 7.2devi15 cet ornement se constitue de cinq plaques en argent , attachés les uns avec les autres par des anneaux et des calottes hémisphériques émaillées de trois couleurs ( jaune , vert , bleu )tandis que , la plaque centrale est formée d’une seule pièce rectangulaire grande et un peu large surmontée d’un triangle et à la base des plaques , elle pendantes de différentes palettes .",
" - fonction symbolique de tabzimt : ce bijou se fixe généralement sur le front ou la poitrine de la femme mariée ; « quand la femme mariée porte ce bijou sur le côté gauche cela signifie que cette dernière a accouché un garçon .",
" - tableau n°12 : signification detabzimt 7.2.1.2.2 abzim: ce fibule est sous la forme d’une plaque triangulaire , nous trouvons dans leur base deux appendices triangulaires ou circulaires , leur sommet est habituellement surmonté d’une plaque arrondie , émaillée de jaune , vert et bleu sur les deux surfaces et aussi décorées de cabochons de corail .",
" - (figure 04) fonction symbolique de abzim : ce bijou se porte sur la poitrine par les jeunes filles , « quand elle le porte sur le côté gauche cela veut dire son indisponibilité pour l’engagement ou elle est fiancé , et à l’inverse quand elle le porte sur le côté droite cela signifie qu’elle est célibataire »54.",
" - tableau n °13 : signification de abzimboucles d’oreilles figure 5 figure 6 7.2.1.3 boucles d’oreilles: les boucles d’oreilles sont des bijoux qui se po eux plaqu ses cent il , les de au , émaa ngulair une pende es plus utre de lafonction symbolique de ce genre de boucles d’oreilles : ce genre de boucles est porté par les jeune filles , cela signifie « qu’elle est prête pour le mariage et pour avoir des enfants .",
" - »55 tableau n°14 : signification de boucles d’oreilles 7.2.1.3.1 letrak: sorte des boucles d’oreilles très anciennes qui présente par un anneau ovale orné à l’extrémité par des sertissage de corail et d’émaux .",
" - fonction symbolique de ddah : ce forme de bijou il est pour la mariée , c’est le seul bracelet « qu’elle porte dans le jour de son mariage .»58tableau n°17 : signification de ddah chevillière 7.2.1.5 chevillière: il s’agit d’une grande chaine de cheville , leur hauteur est 11cm faites entièrement de haute qualité en argent agrémentée de trois cabochons de corail rouge , ce bracelet de cheville est ouvert et sur les deux côtés de l’ouverture il y a une boucle où la fermeture a été fixé , caractérisé par l’absence de décor émaillé , il s’appelle axelxal .fonction symbolique de axelxal : cette sorte de bracelet porte plusieurs message tout dépond de la femme qui le porte , parmi ces messages :  « quand la femme mariée porte khelkhal à ses chevilles cela veut dire qu’elle n’est plus célibataire.",
" - ce collier émaillé ques et des pendeloques , nous trfonction symbolique de taziba : taziba a plusieurs significations parmi lesquelles : « la tristesse , la colère ; selon la légende la femme qui le porte est devenue veuve suite à l’assassinat de son mari .»60 tableau n°19 : signification de taziba figure 117.2.1.7 ceinture: dans la kabylie , les ceintures étaient faites de fils d’argent de différentes couleurs , elle se compose de 14 plaques émaillées rectangulaire auxquelles s’ajoutent les deux plaques centrales qui sont fermées par une goupille à tête , chaque laque porte en son centre une sertissure de corail.",
" - tableau n°20 : signification de la ceinture7.3 les bijoux nailis: diadème figure 12 7.3.1 nassya ( aassaba ) : ainsi on l’appelle chez oulednail , c’est un bande d’argent de longueur et de largeur fixe , composé d’un ensemble de pièces ( 07 pièces ) qui se terminent par des panneaux ajustés avec une corles n centre avec d gulaires lobés .",
" - fonction symbolique de kholala : « kholala est utilisée par la jeune mariée associés à des rangs de clous de girofle , ainsi que la présence de substances odoriférantes est sensé gratifier le couple nouvellement formé de protection et faveur .»62 tableau n°22 : signification de kholalaboucles d’oreilles figure 15 7.3.3 boucles d’ 7.3.3.1 mcharee il est considéré constitué d' anneaux qui portent dure, soient des rangées croiséeset là , dans l’orn ues pour la décoration de sa par nailiyates.",
" - »63 tableau n°23 : signification du mchareef figure 167.3.4 bracelet : bracelet à tiges ou bien megouassa bien qu’origine de la tribu nomade saharienne d' oulednail , est l’un des bijoux nailis les plus important et distinctif , en argent ajouré avec incrustation de corail , d’une largeur de 4.5 cm et de diamètre 6.5 cm .",
" - »64  « pour permettre à la femme nailie de se défendre contre tout danger au quelle est confronté .»65 tableau n°24 : signification demegoussachevillière figure 17 7.3.5 chevillière : il existe deux types de khelkhel : une chevillière avec un sonnet c’est-à-dire qu’il émet un son lorsqu’il se déplace , l’autre type c’est sans sonnet c’est dire qu’il muet lorsqu’il se déplace.",
" - tableau n°27 : signification de skhabceinture figure 19 7.3.7 mehazma : ce sont des pièces et des unités rectangulaire en argent , décorées et reliées les unes aux autres dans son centre il y a une grande figure décorée et elle est différente des autres unités qui formée cette ceinture et sous cette forme nous trouvons la clé de la ceinture .parmi les ceintures les plus importantes chez oulednail nous trouvons : la ceinture de khomssa et croisant , et la ceinture de soleil et d’étoile ."

]

summaries = []
text = " ".join(sentences)
for text in sentences[:4]:

  preprocess_text = text.strip().replace("\n","")
  input_ids = tokenizer("summarize: " + preprocess_text, return_tensors="pt").input_ids  # Batch size 1
  outputs = model.generate(input_ids)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  summaries.append(summary)
summaries